In [3]:
import numpy as np
import pandas as pd
import sklearn
import mglearn
from sklearnex  import  patch_sklearn 
patch_sklearn ()


dataset = pd.read_csv('dataset_limpio.csv')
dataset = dataset.drop(['Unnamed: 0'], axis=1)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


array([[4.04500000e+03, 2.00000000e+00, 5.00000000e+00, ...,
        1.00000000e+00, 2.79717007e-01, 1.00000000e+00],
       [2.84000000e+02, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 3.54455658e-01, 1.00000000e+00],
       [4.30200000e+03, 2.00000000e+00, 5.00000000e+00, ...,
        1.00000000e+00, 2.81260355e-01, 1.00000000e+00],
       ...,
       [3.73100000e+03, 1.00000000e+00, 3.00000000e+00, ...,
        0.00000000e+00, 3.52280885e-01, 1.00000000e+00],
       [2.05910000e+04, 0.00000000e+00, 2.00000000e+00, ...,
        0.00000000e+00, 3.04800991e-01, 1.00000000e+00],
       [3.03900000e+03, 1.00000000e+00, 3.00000000e+00, ...,
        0.00000000e+00, 3.43809795e-01, 1.00000000e+00]])

In [ ]:
from sklearnex  import  patch_sklearn 
patch_sklearn ()
from time import time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors


Y = dataset[:50000]
reduced_data = PCA(n_components = 0.95).fit_transform(dataset)

def bench_marks(algoritm, name):
    
    t0 = time()
    neigh = NearestNeighbors()
    neigh.fit(dataset.values)
    
    X = neigh.radius_neighbors_graph(dataset.values, mode = 'distance')
    
    print(X)
    estimator = make_pipeline(StandardScaler(), algoritm).fit(X)
    labels= estimator[-1].labels_
        
    fit_time = time() - t0
    results = [name, fit_time]
    
    
    results += [silhouette_score(X, labels, metric = 'euclidean')]
    results += [davies_bouldin_score(X,labels)]
    results += [calinski_harabasz_score(X, labels)]
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    results += [n_clusters]
    
    formatter_result = (
        "{:6s}\t\t{:.3f}s\t{:.3f}\t\t{:.3f}\t\t{:.2f}\t\t{:.0f}"
    )
    print(formatter_result.format(*results))

In [ ]:
from sklearn.cluster import DBSCAN

from sklearnex  import  patch_sklearn 
patch_sklearn ()


print(82 * "_")
print("init\t\t\ttime\t\tsilhouette\tdavies_bouldin\tcalinski_harabasz\tn_clusters\n")


# db = DBSCAN(eps = 10, min_samples = 50)
# bench_marks(db,name = "db")

# db = DBSCAN(eps = 100, min_samples = 50)
# bench_marks(db,name = "db")

db = DBSCAN(eps = 0.00001, min_samples = 300000, metric = 'precomputed',  n_jobs = -1)
bench_marks(db,name = "db")


# db = DBSCAN(eps = 10, min_samples = 4000) #con estos parametros salen 8 clusters pero un score de -0.395a silhouette y 1.791 davies
# bench_marks(db, name="db")

print(82 * "_")